## Steps
1. Read csv file as pandas dataframe/Create cost matrix
  * Add duplicate columns
  * assign lottery number to doctors (higher lottery number, less likely doctor is unassigned
  * add dummy positions
2. Column and row reduction - For each row, find the minimum value and subtract it from all elements in that row. Next, do the same for each column.
3. Draw the minimum number of horizontal and vertical lines to cover all the zeros in the matrix.
4. Find the smallest uncovered element, which is 1.

  * Subtract 1 from all uncovered elements.

  * Add 1 to all elements at the intersection of lines (none in this case).

  * Keep all other elements the same.
5. Now, draw the minimum number of lines to cover all zeros in this new matrix.
6. Find an assignment of zeros such that no two assignments share a row or column.

    * Note: if there are more residency programs than there are unique rankings, then a lottery system must be established to assign lottery numbers to each person, which gives different cost to each applicant, which allows for the algorithm to choose. The lottery number will be the dummy number of the doctor. The lottery number can also be calculated via GPA, and other stats.


In [2]:
#dependencies
import requests
import io
from io import StringIO
from google.colab import files
import pandas as pd
import numpy as np

In [4]:
#Lillian
def import_from_github(github_url):
  """
  Allow user to import a github url as csv
  """
  #if a regular url
  if 'github.com' in github_url and 'raw.githubusercontent.com' not in github_url:
    github_url= github_url.replace('github.com', 'raw.githubusercontent.com')
    github_url= github_url.replace('/blob', '/')

  if not github_url.endswith('.csv'):
    print("URL does not point to a CSV file")

  df= pd.read_csv(github_url)
  return df

In [5]:
#Lillian
def import_from_local():
  """
  Allow user to import a local csv file
  """
  print("Please select a CSV file to upload:")
  uploaded = files.upload()

  if not uploaded:
    print("Error: No file selected")
    return None

  filename = list(uploaded.keys())[0]


  if not filename.lower().endswith('.csv'):
    print("Error: File is not a CSV file")
    return None

  df = pd.read_csv(io.BytesIO(uploaded[filename]))
  df = pd.read_csv(filename)
  return df

In [6]:
#Lillian
#UI to import files
def import_csv_files(num_files=3):
  print(f'CSV File Importer for Google Colab - Import {num_files} Files\n')
  print(f'Please Import Doctor-Residency Rankings, Number of Positions per Residency, and/or Doctor Statisitcs')
  print('=====================================================================================================')

  dataframes = []
  for i in range(num_files):
    print('\n')
    print(f'Importing File {i+1} of {num_files}...')
    print(f'File {i+1}:')
    choice= input('Enter your choice (1 or 2):').strip()
    print('1. Import from GitHub URL')
    print('2. Import from Local File')
    print('\n')

    if choice == '1':
      url= input('Enter GitHub URL:').strip()
      df= import_from_github(url)
      if df is not None:
        dataframes.append(df)
      else:
        print(f'Failed to import file {i+1}. Invalid GitHub URL.')
    elif choice == '2':
      df= import_from_local()
      if df is not None:
        dataframes.append(df)
      else:
        print(f'Failed to import file {i+1}.')
    else:
      print('Invalid choice. Please enter 1 or 2.')
      i-=1
  return dataframes

In [7]:
dataframes = import_csv_files(num_files=3)

CSV File Importer for Google Colab - Import 3 Files

Please Import Doctor-Residency Rankings, Number of Positions per Residency, and/or Doctor Statisitcs


Importing File 1 of 3...
File 1:
Enter your choice (1 or 2):2
1. Import from GitHub URL
2. Import from Local File


Please select a CSV file to upload:


Saving 100_doctors_10_residency_programs - Sheet1.csv to 100_doctors_10_residency_programs - Sheet1.csv


Importing File 2 of 3...
File 2:
Enter your choice (1 or 2):2
1. Import from GitHub URL
2. Import from Local File


Please select a CSV file to upload:


Saving 100_doctor_stats - Sheet1.csv to 100_doctor_stats - Sheet1.csv


Importing File 3 of 3...
File 3:
Enter your choice (1 or 2):2
1. Import from GitHub URL
2. Import from Local File


Please select a CSV file to upload:


Saving num_positions - Sheet1.csv to num_positions - Sheet1.csv


In [8]:
dataframes[0]

,Name,Mayo Clinic College of Medicine and Science,University of California San Francisco,Vanderbilt University Medical Center,McGaw Medical Center of Northwestern University,Pennsylvania Hospital of Penn Medicine,University of Texas Southwestern Medical Center,New York-Presbyterian/Columbia University Irving Medical Center,Massachusetts General Hospital,Duke University Hospital,Baylor College of Medicine
0,Dee Gibbs,3,2,5,7,9,1,10,8,4,6
1,Carolyn Arnold,10,1,2,3,4,5,6,7,8,9
2,Savannah Daugherty,8,5,10,4,2,3,7,6,1,9
3,Millie Serrano,2,9,4,1,10,8,7,6,3,5
4,Christina Smith,9,2,10,6,7,8,3,4,1,5
...,...,...,...,...,...,...,...,...,...,...,...
95,Evangelina Nolan,3,9,8,2,7,1,5,4,10,6
96,Ashley York,10,3,2,8,7,6,4,5,9,1
97,Yolanda House,2,5,6,9,7,4,8,1,10,3
98,Annette Solomon,9,6,8,2,1,7,3,4,5,10


In [9]:
dataframes[1]

,Name,USMLE,GPA,Clinical Experience,Letter of Recommendation,Interpersonal Qualities
0,Dee Gibbs,275,3.08,10,4,5
1,Carolyn Arnold,276,3.68,7,4,5
2,Savannah Daugherty,208,3.94,5,9,8
3,Millie Serrano,220,3.62,9,8,5
4,Christina Smith,291,3.05,7,9,7
...,...,...,...,...,...,...
95,Evangelina Nolan,201,3.89,5,9,4
96,Ashley York,295,3.05,10,9,4
97,Yolanda House,240,3.91,10,6,8
98,Annette Solomon,280,3.53,8,10,5


In [10]:
dataframes[2]

,Unnamed: 0,Mayo Clinic College of Medicine and Science,University of California San Francisco,Vanderbilt University Medical Center,McGaw Medical Center of Northwestern University,Pennsylvania Hospital of Penn Medicine,University of Texas Southwestern Medical Center,New York-Presbyterian/Columbia University Irving Medical Center,Massachusetts General Hospital,Duke University Hospital,Baylor College of Medicine
0,Positions,10,9,7,6,13,18,4,15,3,5


In [11]:
#Lillian
def lottery_numbers(df, stats_df=None):
  """
  Caluclate lottery numbers based on normalized scores across all columns.
  If no stats are provided, assign random lottery numbers.

  Imput:
  df (Dataframe) - Doctor-Residency Rankings dataframe
  stats_df (Dataframe, optional) - Doctor stats dataframe with numerical values to be normalized


  Return:
  Series: Lottery numbers where lower number indicate better ranking
  """
  if stats_df is None:
    random_num = np.random.permutation(df.shape[0])
    return pd.Series(random_num) + df.shape[1]
  else:
    #drop the first column because those are the names of the doctor
    stats_norm = stats_df.copy().iloc[:, 1:]

    for column in stats_norm.columns:
      col_min= stats_norm[column].min()
      col_max= stats_norm[column].max()

      # if all values in columns are the same
      if col_max== col_min:
        stats_norm[column] = 1.0/len(stats_norm.columns)
      else:
        #We assume that schools weight each of the qualities equally (want a well arounded canidate)
        #Ex. GPA is worth 1/4th of your score
        #We normalize the values in each column to be between 0 and 1 and divide by number of columns
        stats_norm[column] = ((stats_norm[column] - col_min)/ (col_max - col_min))/len(stats_norm.columns)

      stats_rank= stats_norm.sum(axis=1)
      #lottery number to assign doctors (lower the number, the more likely they are to get in dummy position)
      #if doctors have same stats, then the doctor who comes first will have will be ranked first
      stats_rank= stats_rank.rank(ascending=True, method='first')
      stats_rank= [num + (df.shape[1]-1) for num in stats_rank]
      lottery_num= pd.Series(stats_rank).astype(int)

    return lottery_num

In [24]:
#Lillian
def matrix_prep(df, residency_positions= [], lottery_num= []):
  """
  Prepare the matrix for the Hungarian algorithm by duplicating columns
  for multiple positions and adding dummy positions with lottery numbers.

  Input:
  df (DataFrame) - Original dataframe of program preferences
  residency_positions (list) - List of positions available for each program
  lottery_num (Series/list) - Lottery numbers for each doctor

  Return:
  DataFrame - Prepared matrix with duplicated columns and dummy positions.
  """

  #number of dummy positions
  dummy_positions = df.shape[0]- sum(residency_positions)


  original_columns = list(df.columns)
  new_columns_order = []

  #duplicate the columns
  for program, positions in zip(df.columns[1:], residency_positions):
    new_columns_order.append(program)
    for position in range(2, positions+1):
        duplicate_col_name= f'{program}_{position}'
        new_columns_order.append(duplicate_col_name)
        df[duplicate_col_name] = df[program]

  df = df[new_columns_order]

  #add dummy rows
  for i in range(1, dummy_positions+1):
    df[f'Dummy_{i}']= 0

  # assign the lottery number in the dummy position
  for column in range(1, dummy_positions+1):
    df.iloc[:, -column] = lottery_num

  return df


Hard-coded matrix manipulation


1. Column and row reduction - For each row, find the minimum value and subtract it from all elements in that row. Next, do the same for each column.

3. Build a bipartite graph where edges represent zeros and we try to traverse the graph efficiently. The goal is to find an augmenting path that checks if the doctor can be matched to a residency slot. If the slot is taken, it recursively attempts to find the next available slot.


In [13]:
#Alex
def hungarian_matrix_reduction(df):
  '''
  Performs row and column reduction to create matrix with at least one zero in each row and column.
  '''

  matrix = df.to_numpy()

  #row reduction
  row_min = np.min(matrix, axis=1).reshape(-1, 1)
  row_reduced_matrix = matrix - row_min

  #column reduction
  col_min = np.min(row_reduced_matrix, axis=0).reshape(1, -1)
  reduced_matrix = row_reduced_matrix - col_min

  return reduced_matrix

In [14]:
#Alex
def minimum_lines(reduced_matrix):
  '''
  Finds the minimum number of lines to cover all zeros in the matrix.
  Uses bipartite matching to deteemine if the current matrix contains a complete assignment.
  If not, it determines how many lines are needed to cover all zeros.


  Imput:
  reduced_matrix

  Output: result: int that specificies minimum number of lines to cover all zeros
  matchR: list where indices represent residency and value is the doctor matched to it (-1 if unmatched)

  '''
  num_rows, num_cols = reduced_matrix.shape

  #builds bipartite graph where edges represent zeros
  graph = {} # Creates a set of lists, where each element of the list is the row, and the contents of the list are the columns in that row which have a zero.
  for i in range(num_rows):
    graph[i] = []
    for j in range(num_cols): # In each column, if there is a 0 for that row, append it to the list in the index of the graph set.
      if reduced_matrix[i, j] == 0:
        graph[i].append(j)

  #bipartite depth search
  def bpm(bpGraph, u, seen, matchR): # function defined within a function
    '''bpGraph = dict where each list in a dictionary represents residencies a doctor can match to,
    u = current doctor,
    seen = T/F depending on whether residency positions have been visited during search,
    matchR = list where indices represent residency and value is the doctor matched to it (-1 if unmatched)'''
    for v in bpGraph[u]: # for residency possible for the doctor to be matched with
        if not seen[v]: # if the residency has not been seen and checks for existence of an edge
            seen[v] = True # change that that residency has been seen
            if matchR[v] < 0 or bpm(bpGraph, matchR[v], seen, matchR): # if doctor is unmatched to residency or the currently matched doctor can be reassigned to a different residency
                # attempts to find an augmenting path from already-matched doctor in order to iteratively optimize assignment
                matchR[v] = u # doctor takes that slot in the residency
                return True # return that the Doctor has been matched
    return False # Return the doctor is not matched

  # matchR: for every column, we see which row is matched to it (-1 if unmatched)
  matchR = [-1] * num_cols
  result = 0

  for u in range(num_rows):
      seen = [False] * num_cols
      if bpm(graph, u, seen, matchR):
          result += 1

  return result, matchR

In [16]:
#Lillian
def cover_zeros_efficiently(reduced_matrix):
  """
  Finds the minimum vertex cover using Konig's theorem.
  Identifies which rows and columns to cover to eliminate all zeros with minimum number of lines.
  """

  num_rows, num_cols = reduced_matrix.shape

  _, matchR= minimum_lines(reduced_matrix)

  #finding unmatched rows
  unmatched_rows=set()
  row_matches= [-1]*num_rows

  #we build row_matches from the column matches
  for i, j in enumerate(matchR):
    if i != -1:
      row_matches[i] = j

  for i in range(num_rows):
    if row_matches[i] == -1:
      unmatched_rows.add(i)

  visited_rows=set()
  visited_cols=set()
  queue= list(unmatched_rows)

  while queue:
    #get ride of the first in queue
    # u is current row
    u= queue.pop(0)
    visited_rows.add(u)

    #find all zeros in this row
    for v in range(num_cols):
      if reduced_matrix[u, v]== 0 and v not in visited_cols:
        visited_cols.add(v)

        #find the row matched to the column
        matched_row=matchR[v]
        if matched_row != -1 and matched_row not in visited_rows:
          queue.append(matched_row)

    #implementation of Konig's theorem
    # the minimum vertex cover is equal to unmatched row + columns reachable from unmatched rows
    covered_rows= set(range(num_rows))- visited_rows
    covered_cols= visited_cols

  return covered_rows, covered_cols # returns tuple of sets ({},{}})

In [74]:
# Amos
# define a function that does the following
''' 1. go thru the matrix and find uncovered values: subtract the minimum (k) from these values
2. if a value exists at the intersection of a row and column, add the minimum value (k) to it
3. reset the system and run the algorithm again'''

def subtract_min(matrix, covered_rows, covered_cols):
  # find k
    matrix_after_row_removal = np.delete(matrix, list(covered_rows), axis=0)
    matrix_after_col_removal = np.delete(matrix_after_row_removal, list(covered_cols), axis=1)

    # Check if the matrix after removal is empty
    if matrix_after_col_removal.size == 0:
        return matrix

    # Find the minimal uncovered value
    k = np.min(matrix_after_col_removal)

    # Adjust the original matrix
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            if i in covered_rows and j in covered_cols:
                matrix[i, j] += k  # Add k to double-covered cells
            elif (i in covered_rows and j not in covered_cols) or (i not in covered_rows and j in covered_cols):
                continue  # Skip single-covered elements
            else:
                matrix[i, j] -= k  # Subtract k from uncovered cells

    return matrix

In [79]:
#Amos
from scipy.optimize import linear_sum_assignment



def assign_jobs(matrix, residency_position):
    '''
    Inputs: matrix: square matrix of data
    residency_position: list of number of positions per residency

    outputs: assignments: list of tuples (doctor_index, residency_index)
    '''

    matrix = np.array(matrix, dtype=float)

    total_residency_positions = sum(residency_position)
    num_dummy_columns = matrix.shape[1] - total_residency_positions

    row_indices, col_indices = linear_sum_assignment(matrix)
    assignments = []
    for doctor_idx, residency_idx in zip(row_indices, col_indices):
        if residency_idx >= total_residency_positions:
            # Assign to dummy columns if the residency index corresponds to a dummy
            assignments.append((doctor_idx, -1))
        else:
            # Assign to actual residency programs
            assignments.append((doctor_idx, residency_idx))

    return assignments

In [90]:

def organize_assignments(assignments, residency_positions: list, doctor_names=None, residency_names=None):
    ''' The point of this function is to translate slots back into residencies
    Inputs: assignments: list of tuples
    residency_positions: list of number of positions per residency
    doctor_name: name of the doctors
    residency_names: name of the residencies
    '''
    # Mapping logic
    residency_map = []
    if residency_names is None:
        residency_names = [f'Residency {i+1}' for i in range(len(residency_positions))]
    # Create a mapping of each slot's numerical index to its corresponding residency name.
    # e.g., with [5, 10], the first 10 slots map to 'Residency 1', and the next 5 map to 'Residency 2'.
    for index, number_of_positions in enumerate(residency_positions):
        for _ in range(number_of_positions):
            residency_map.append(residency_names[index])

    # Print the assignments with residency names like in previous function:
    print("Assignments (Doctor Index -> Residency Name):")
    print("\n")

    for doctor_idx, slot_idx in assignments:
        # use doctor name if provided
        doctor_name= doctor_names[doctor_idx] if doctor_names else f"Doctor {doctor_idx}"
        if slot_idx == -1:
            print(f"{doctor_name} -> Unassigned")
        else:
            try:
                residency_name = residency_map[slot_idx]
                print(f"{doctor_name} -> {residency_name}")
            except IndexError:
                print(f"Doctor {doctor_idx} -> Error: Invalid Slot {slot_idx}")

    return residency_map # Return the map for further use if needed

In [91]:
def execute(rank_df, stats_df, residency_position_df):
  lottery_num = lottery_numbers(rank_df, stats_df)

  residency_position = residency_position_df.iloc[0, 1:].tolist()

  df = matrix_prep(rank_df, residency_position, lottery_num)
  reduced = hungarian_matrix_reduction(df)
  # one iteration, but continues until optimal solution is found
  iteration = 0
  max_iterations = 50  # safety limit

  while iteration < max_iterations:
      lines, _ = minimum_lines(reduced)
      if lines == len(rank_df):
          break

      #covering lines and improve matrix
      covered_rows, covered_cols = cover_zeros_efficiently(reduced)
      reduced = subtract_min(reduced, covered_rows, covered_cols)
      iteration += 1

  assignments = assign_jobs(reduced, residency_position)

  doctor_names = rank_df.iloc[:, 0].tolist()

  #residency names from the original dataframe columns (excluding Name column)
  residency_names = rank_df.columns[1:].tolist()

  final_assignments = organize_assignments(assignments, residency_position, doctor_names, residency_names)

  return final_assignments


In [92]:
residency_map = execute(dataframes[0], dataframes[1], dataframes[2])


Assignments (Doctor Index -> Residency Name):


Dee Gibbs -> University of Texas Southwestern Medical Center
Carolyn Arnold -> University of California San Francisco
Savannah Daugherty -> Pennsylvania Hospital of Penn Medicine
Millie Serrano -> Mayo Clinic College of Medicine and Science
Christina Smith -> University of California San Francisco
Evangelina Stein -> University of Texas Southwestern Medical Center
Deann Bell -> Pennsylvania Hospital of Penn Medicine
Hobert Long -> Duke University Hospital
Laurel Cardenas -> Unassigned
Will Cooley -> University of Texas Southwestern Medical Center
Gay York -> Duke University Hospital
Sherri Shields -> University of California San Francisco
Susanne Parker -> Unassigned
Lucinda Lowe -> McGaw Medical Center of Northwestern University
Elbert Dorsey -> Unassigned
Marisol Pearson -> Pennsylvania Hospital of Penn Medicine
Weston Dunn -> Unassigned
Amanda Mcneil -> Massachusetts General Hospital
Gary Castillo -> Massachusetts General Hospital
Raym